## Project: Predicting Boston Housing Prices
In this project, we will evaluate the performance and predictive power of a model that has been trained and tested on data collected from homes in suburbs of Boston, Massachusetts. A model trained on this data that is seen as a *good fit* could then be used to make certain predictions about a home — in particular, its monetary value. This model would prove to be invaluable for someone like a real estate agent who could make use of such information on a daily basis.

The dataset for this project originates from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/Housing). The Boston housing data was collected in 1978 and each of the 506 entries represent aggregated data about 14 features for homes from various suburbs in Boston, Massachusetts. For the purposes of this project, the following preprocessing steps have been made to the dataset:
- 16 data points have an `'MEDV'` value of 50.0. These data points likely contain **missing or censored values** and have been removed.
- 1 data point has an `'RM'` value of 8.78. This data point can be considered an **outlier** and has been removed.
- The features `'RM'`, `'LSTAT'`, `'PTRATIO'`, and `'MEDV'` are essential. The remaining **non-relevant features** have been excluded.
- The feature `'MEDV'` has been **multiplicatively scaled** to account for 35 years of market inflation.

## Getting Started
Run the code cell below to load the Boston housing dataset, along with a few of the necessary Python libraries required for this project. We will know the dataset loaded successfully if the size of the dataset is reported.

In [1]:
# Import libraries necessary
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import *

# Import supplementary visualizations code visuals.py
import visuals as vs

# Pretty display for notebooks
%matplotlib inline

# Initialize Spark and SQL Contexts
spark_context = SparkContext()
sc = SQLContext(spark_context)

# Load the Boston housing dataset
data = sc.read.csv('housing.csv', header=True, inferSchema=True)
prices = data.select('MEDV')
features = data.drop('MEDV')

# Success
print("Boston housing dataset has {} data points with {} variables each.".format(data.count(), len(data.columns)))

Boston housing dataset has 489 data points with 4 variables each.


## Data Exploration
In this first section of this project, we will make a cursory investigation about the Boston housing data and provide our observations. Familiarizing yourself with the data through an explorative process is a fundamental practice to help better understand and justify the results.

Since the main goal of this project is to construct a working model which has the capability of predicting the value of houses, we will need to separate the dataset into **features** and the **target variable**. The **features**, `'RM'`, `'LSTAT'`, and `'PTRATIO'`, give us quantitative information about each data point. The **target variable**, `'MEDV'`, will be the variable we seek to predict. These are stored in `features` and `prices`, respectively.

### Implementation: Calculate Statistics
We will calculate descriptive statistics about the Boston housing prices. These statistics will be extremely important later on to analyze various prediction results from the constructed model.

In the code cell below, we have implemented the following:
- Calculated the minimum, maximum, mean, median, and standard deviation of `'MEDV'`, which is stored in `prices`.
  - Stored each calculation in their respective variable.

In [2]:
# Minimum price of the data
minimum_price = prices.agg({'MEDV':'min'}).collect()[0]['min(MEDV)']

# Maximum price of the data
maximum_price = prices.agg({'MEDV':'max'}).collect()[0]['max(MEDV)']

# Mean price of the data
mean_price = prices.agg({'MEDV':'avg'}).collect()[0]['avg(MEDV)']

# Median price of the data
median_price = prices.approxQuantile('MEDV', [0.5], 0)[0]

# Standard deviation of prices of the data
std_price = prices.agg({'MEDV':'stddev_pop'}).collect()[0]['stddev_pop(MEDV)']

# Show the calculated statistics
print("Statistics for Boston housing dataset:\n")
print("Minimum price: ${:,.2f}".format(minimum_price))
print("Maximum price: ${:,.2f}".format(maximum_price))
print("Mean price: ${:,.2f}".format(mean_price))
print("Median price ${:,.2f}".format(median_price))
print("Standard deviation of prices: ${:,.2f}".format(std_price))

Statistics for Boston housing dataset:

Minimum price: $105,000.00
Maximum price: $1,024,800.00
Mean price: $454,342.94
Median price $438,900.00
Standard deviation of prices: $165,171.13


### Question 1 - Feature Observation
As a reminder, we are using three features from the Boston housing dataset: `'RM'`, `'LSTAT'`, and `'PTRATIO'`. For each data point (neighborhood):
- `'RM'` is the average number of rooms among homes in the neighborhood.
- `'LSTAT'` is the percentage of homeowners in the neighborhood considered "lower class" (working poor).
- `'PTRATIO'` is the ratio of students to teachers in primary and secondary schools in the neighborhood.


** Using your intuition, for each of the three features above, do you think that an increase in the value of that feature would lead to an **increase** in the value of `'MEDV'` or a **decrease** in the value of `'MEDV'`? Justify your answer for each.**

**Hint:** This problem can be phrased using examples like below.  
* Would you expect a home that has an `'RM'` value(number of rooms) of 6 be worth more or less than a home that has an `'RM'` value of 7?
* Would you expect a neighborhood that has an `'LSTAT'` value(percent of lower class workers) of 15 have home prices be worth more or less than a neighborhood that has an `'LSTAT'` value of 20?
* Would you expect a neighborhood that has an `'PTRATIO'` value(ratio of students to teachers) of 10 have home prices be worth more or less than a neighborhood that has an `'PTRATIO'` value of 15?

**Answer: **

**'RM':** More number of rooms mean high price, because the area of the plot on which the house is may be more than the one with lesser rooms. The designing and construction of the house would also have cost more, which justifies the high price.

**'LSTAT':** The higher the percentage of lower class homeowners, the lesser the price of the house. This is because the prices of the neighbouring houses also have an impact on the price of the house based on demand and the class of the people trying to buy the house in that area.

**'PTRATIO':** Lower PTRATIO could signify good education settings in the nearby schools, which increases the demand of such areas and the houses in these areas, which means an increase in the price.

----

## Developing a Model
In this second section of the project, we have developed the tools and techniques necessary for a model to make a prediction. Being able to make accurate evaluations of each model's performance through the use of these tools and techniques helps to greatly reinforce the confidence in our predictions.

### Implementation: Define a Performance Metric
It is difficult to measure the quality of a given model without quantifying its performance over training and testing. This is typically done using some type of performance metric, whether it is through calculating some type of error, the goodness of fit, or some other useful measurement. For this project, we have calculated the [*coefficient of determination*](http://stattrek.com/statistics/dictionary.aspx?definition=coefficient_of_determination), R<sup>2</sup>, to quantify your model's performance. The coefficient of determination for a model is a useful statistic in regression analysis, as it often describes how "good" that model is at making predictions. 

The values for R<sup>2</sup> range from 0 to 1, which captures the percentage of squared correlation between the predicted and actual values of the **target variable**. A model with an R<sup>2</sup> of 0 is no better than a model that always predicts the *mean* of the target variable, whereas a model with an R<sup>2</sup> of 1 perfectly predicts the target variable. Any value between 0 and 1 indicates what percentage of the target variable, using this model, can be explained by the **features**. _A model can be given a negative R<sup>2</sup> as well, which indicates that the model is **arbitrarily worse** than one that always predicts the mean of the target variable._

For the `performance_metric` function in the code cell below, we have implemented the following:
- Used `r2` from `RegressionMetrics` in the `pyspark.mllib.evaluation` module to perform a performance calculation between `true_and_predicted`.
- Assigned the performance score to the `score` variable.

In [3]:
# Import 'RegressionMetrics'
from pyspark.mllib.evaluation import RegressionMetrics


def performance_metric(true_and_predicted):
    """ Calculates and returns the performance score between 
        true and predicted values based on the metric chosen. """
    
    # Calculate the performance score between 'true' and 'predictions'
    score = RegressionMetrics(true_and_predicted.rdd.map(lambda row: (row.true, row.predictions))).r2
    
    # Return the score
    return score

### Question 2 - Goodness of Fit
Assume that a dataset contains five data points and a model made the following predictions for the target variable:

| True Value | Prediction |
| :-------------: | :--------: |
| 3.0 | 2.5 |
| -0.5 | 0.0 |
| 2.0 | 2.1 |
| 7.0 | 7.8 |
| 4.2 | 5.3 |

Run the code cell below to use the `performance_metric` function and calculate this model's coefficient of determination.

In [4]:
# Calculate the performance of this model
sample_vals = list(zip([3.0, -0.5, 2.0, 7.0, 4.2], [2.5, 0.0, 2.1, 7.8, 5.3]))
sample_data = sc.createDataFrame(sample_vals, ['true_vals', 'predictions'])
score = performance_metric(sample_data.selectExpr('true_vals as true', 'predictions'))
print("Model has a coefficient of determination, R^2, of {:.3f}.".format(score))

Model has a coefficient of determination, R^2, of 0.936.


* Would you consider this model to have successfully captured the variation of the target variable? 
* Why or why not?

** Hint: **  The R2 score is the proportion of the variance in the dependent variable that is predictable from the independent variable. In other words:
* R2 score of 0 means that the dependent variable cannot be predicted from the independent variable.
* R2 score of 1 means the dependent variable can be predicted from the independent variable.
* R2 score between 0 and 1 indicates the extent to which the dependent variable is predictable. An 
* R2 score of 0.40 means that 40 percent of the variance in Y is predictable from X.

**Answer:**

A high R2 score of 0.936 for the model indicates that it has successfully captured 93.6% of the variation of the target variable.

### Implementation: Shuffle and Split Data
Our next implementation requires that we take the Boston housing dataset and split the data into training and testing subsets. Typically, the data is also shuffled into a random order when creating the training and testing subsets to remove any bias in the ordering of the dataset.

For the code cell below, we have implemented the following:
- Used `randomSplit` to shuffle and split the `features` and `prices` data into training and testing sets.
  - Split the data into 80% training and 20% testing.
  - Set the `seed` to a value of our choice. This ensures results are consistent.
- Assigned the train and testing splits to `X_train`, `X_test`, `y_train`, and `y_test`.

In [5]:
# Shuffle and split the data into training and testing subsets
X_train, X_test = data.randomSplit([0.8, 0.2], 7777777)
y_train = X_train.select('MEDV')
y_test = X_test.select('MEDV')
X_train = X_train.drop('MEDV')
X_test = X_test.drop('MEDV')

# Success
print("Training and testing split was successful.")

Training and testing split was successful.


### Question 3 - Training and Testing

* What is the benefit to splitting a dataset into some ratio of training and testing subsets for a learning algorithm?

**Hint:** Think about how overfitting or underfitting is contingent upon how splits on data is done.

**Answer: **

The benefit of splitting a dataset into some ratio is to train a model on the larger portion and to use the smaller portion as unseen data. This said portion of unseen data is used for testing the models performance. Using all of the data for training could result in models which might overfit or underfit on the unseen data. Therefore, we ensure that the model does good using a portion of the data we have for testing.

In [6]:
spark_context.stop()